In [1]:
from IPython.display import display, clear_output, Markdown
import ipywidgets as widgets
from widgets.upload import get_upload_widget
from tagging.sugest import sugerir_rotulos
from extract import extract_data
import io

from database import FinanceDB
import pandas as pd

In [ ]:
import ipywidgets as widgets
from IPython.display import display, Markdown, clear_output
from widgets.new_entry import new_entry_input, mostrar_item
import io


# Função principal de processamento
def process_row(df, index, labels):
    row = df.iloc[index]
    mostrar_item(row, index)

    tipo, categoria, subcategoria, nome = new_entry_input()

    # Sugestão de preenchimento automático
    if rotulos := sugerir_rotulos(row["Data"], row["Valor"], row["Destino / Origem"], row["Descricao"]):
        tipo.value = rotulos.get("Tipo", "")
        categoria.value = rotulos.get("Categoria", "")
        subcategoria.value = rotulos.get("Subcategoria", "")
        nome.value = rotulos.get("Nome", "")

    # Agrupando layout
    linha_superior = widgets.HBox([tipo, categoria, subcategoria], layout=widgets.Layout(gap='10px'))
    linha_inferior = widgets.HBox([nome])
    formulario = widgets.VBox([linha_superior, linha_inferior], layout=widgets.Layout(width='100%', gap='10px'))

    button = widgets.Button(description="Confirmar")
    output = widgets.Output()

    def on_confirm(_):
        label = categoria.value.strip()
        if not label:
            with output:
                clear_output()
                print("Por favor, insira uma categoria antes de confirmar.")
            return

        labels.append(label)

        if index + 1 < len(df):
            process_row(df, index + 1, labels)
        else:
            clear_output()
            print("Fim da lista. Labels cadastrados:")
            for i, lab in enumerate(labels):
                print(f"Row {i}: {lab}")
            exibir_upload_widget()

    button.on_click(on_confirm)
    display(formulario, button, output)


def handle_upload(_):
    if value := upload_widget.value:
        file_info = next(iter(value.values()))
        content = file_info["content"]
        df = extract_data(io.BytesIO(content))
        process_row(df, 0, [])

def exibir_upload_widget():
    global upload_widget
    upload_widget = get_upload_widget()
    upload_widget.observe(handle_upload, names="value")
    display(upload_widget)

# Execução inicial
upload_widget = get_upload_widget()
upload_widget.observe(handle_upload, names="value")
display(upload_widget)


### Item 0
**Data**: 2024-09-25 08:20:13  
**Valor**: -354.16  
**Destino / Origem**: BANCO XP S/A  
**Descricao**: Pagamento para BANCO XP S/A  
**Tipo**: Gasto  


Button(description='Confirmar', style=ButtonStyle())

Output()